In [31]:
import numpy as np
import matplotlib.pyplot as plt

# Q1) Write an RK4 integrator with prototype to take one step. Use $\frac{dy}{dx} = \frac{y}{1+x^2}$, from $x=-20$ to $x=20$, with $y(-20)=1$ using $200$ steps.

In [27]:
def rk4_step(fun,x,y,h):
    k1=fun(x,y)
    k2=fun(x+h/2,y+h*k1/2)
    k3=fun(x+h/2,y+h*k2/2)
    k4=fun(x+h,y+h*k3)
    dy=h*(k1+2*k2+2*k3+k4)/6
    return y+dy
    
def func(x,y):
    dydx=y/(1+x**2)
    return dydx

In [52]:
# Now define linspace and integrate for 200 steps

npt = 201
xx = np.linspace(-20, 20, npt)
h = (xx[1]-xx[0]) # step size
yy = np.zeros(npt)
y0 = 1 # initial condition
yy[0] = y0

for i in range(npt-1): # iterate over all points after initial point, integrating
    yy[i+1] = rk4_step(func,xx[i],yy[i],h)

#fig, ax = plt.subplots(figsize=[14, 8])
integral = h*np.sum(yy)
print(integral)

397.79113254873255


Now write another stepper that takes a step of length $h$, compares that to two steps of $h/2$, and uses them to cancel out the leading-order error term from RK4. How many function evaluations per step does this one use? Use this modified stepper to carry out the same ODE solution using the $\textit{same number of function evaluations}$ as the original. Which is more accurate?

In [48]:
def rk4_stepd(fun,x,y,h):
    upper = rk4_step(fun, x + h/2, y, h/2)
    lower = rk4_step(fun, x, y, h/2)
    big = rk4_step(fun, x, y, h)
    #print("Upper is:" + repr(upper) + "Lower is:" + repr(lower) + 'Big is: ' + repr(big))
    return (upper+lower)/2 - big

In [54]:
diff = 0
for i in range(npt-1): # iterate over all points after initial point, integrating
    diff += rk4_stepd(func,xx[i],yy[i],h)

In [55]:
print(h*np.sum(yy))

397.79113254873255
